
### Autograd: 자동 미분

### Autograd Package

Autograd 패키지는 tensor의 모든 연산에 자동 미분을 제공합니다. 이는 define-by-run의 프레임워크로 코드를 어떻게 작성하느냐에 따라 역전파가 정의된다는 뜻입니다. 역전파는 학습과정의 매 단계마다 달라집니다.

In [1]:
import torch
x = torch.ones(2,2,requires_grad=True) #tensor를 생성하고 requires_grad=True로 연산을 기록합니다.
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [2]:
y = x+2 #gradient function이 자동으로 포함됩니다.
print(y)

z = y*y*3
out = z.mean()
print(z,out)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward1>)


### Gradient

In [3]:

print(out) # out = 3(x+2)*2
out.backward()

print(x)
print(x.grad) # d(out)/dx 를 출력합니다.

tensor(27., grad_fn=<MeanBackward1>)
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [4]:
x = torch.randn(3,requires_grad=True)

y=x*2

while y.data.norm() < 1000:
    
    #data.norm()은 점들 사이의 유클리디안 거리를 나타냅니다
    #torch.sqrt(torch.sum(torch.pow(y, 2)))
    
    y = y*2
    
    print(y,y.data.norm())

tensor([ 1.6321, -2.2635,  0.5391], grad_fn=<MulBackward0>) tensor(2.8422)
tensor([ 3.2643, -4.5270,  1.0782], grad_fn=<MulBackward0>) tensor(5.6844)
tensor([ 6.5285, -9.0541,  2.1565], grad_fn=<MulBackward0>) tensor(11.3688)
tensor([ 13.0571, -18.1082,   4.3130], grad_fn=<MulBackward0>) tensor(22.7376)
tensor([ 26.1142, -36.2164,   8.6259], grad_fn=<MulBackward0>) tensor(45.4751)
tensor([ 52.2284, -72.4328,  17.2519], grad_fn=<MulBackward0>) tensor(90.9502)
tensor([ 104.4568, -144.8656,   34.5038], grad_fn=<MulBackward0>) tensor(181.9004)
tensor([ 208.9135, -289.7312,   69.0075], grad_fn=<MulBackward0>) tensor(363.8008)
tensor([ 417.8271, -579.4623,  138.0151], grad_fn=<MulBackward0>) tensor(727.6017)
tensor([  835.6542, -1158.9247,   276.0302], grad_fn=<MulBackward0>) tensor(1455.2034)


In [5]:
gradients = torch.tensor([0.1,1.0,0.0001],dtype=torch.float)
print(y)
y.backward(gradients)
print(x.grad) # d(y)/d(x) 를 출력합니다

tensor([  835.6542, -1158.9247,   276.0302], grad_fn=<MulBackward0>)
tensor([2.0480e+02, 2.0480e+03, 2.0480e-01])


### 1주차 과제 ))   MNIST 정확도 92% 이상 만들어보기


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim 

from torch.autograd import Variable 
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

batch_size=32
learning_rate=0.06
num_epochs=5

train_dataset=datasets.MNIST(root='./data',train =True, 
                             transform = transforms.ToTensor(), download=True)
test_dataset=datasets.MNIST(root='./data',train =False, 
                             transform = transforms.ToTensor())

train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle = True)
test_loader=DataLoader(test_dataset,batch_size=batch_size,shuffle = False)

In [11]:
class NeuralNetwork(nn.Module):
    def __init__(self,num_classes=10):
        super(NeuralNetwork,self).__init__()
        self.layer1=nn.Linear(28*28,100)
        self.layer2=nn.Linear(100,200)
        self.layer3=nn.Linear(200,num_classes)
        
    def forward(self,x):
        out=x.view(x.size(0),-1)
        out=self.layer1(out)
        out=self.layer2(out)
        out=self.layer3(out)
        return out
#MODEL
model=NeuralNetwork()
#LOSS
criterion = nn.CrossEntropyLoss()
#OPTIMIZER
optimizer = optim.SGD(model.parameters(),lr=learning_rate)
        

In [12]:
#학습시켜주기
for epoch in range(num_epochs):
    for i , (img,label) in enumerate(train_loader,1):
        img,label=Variable(img),Variable(label)
        out=model(img)
        loss=criterion(out,label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(i+1)%100 == 0:
            print('Epoch [{}/{}], Step[{}/{}], Loss :{:.4f}'.format(epoch+1,num_epochs,i+1,len(train_loader),loss.item()))
            

Epoch [1/5], Step[100/1875], Loss :0.9944
Epoch [1/5], Step[200/1875], Loss :0.4209
Epoch [1/5], Step[300/1875], Loss :0.1991
Epoch [1/5], Step[400/1875], Loss :0.4513
Epoch [1/5], Step[500/1875], Loss :0.3930
Epoch [1/5], Step[600/1875], Loss :0.3410
Epoch [1/5], Step[700/1875], Loss :0.9456
Epoch [1/5], Step[800/1875], Loss :0.1546
Epoch [1/5], Step[900/1875], Loss :0.3402
Epoch [1/5], Step[1000/1875], Loss :0.1389
Epoch [1/5], Step[1100/1875], Loss :0.2339
Epoch [1/5], Step[1200/1875], Loss :0.0811
Epoch [1/5], Step[1300/1875], Loss :0.2637
Epoch [1/5], Step[1400/1875], Loss :0.0834
Epoch [1/5], Step[1500/1875], Loss :0.5032
Epoch [1/5], Step[1600/1875], Loss :0.1895
Epoch [1/5], Step[1700/1875], Loss :0.3416
Epoch [1/5], Step[1800/1875], Loss :0.3083
Epoch [2/5], Step[100/1875], Loss :0.3484
Epoch [2/5], Step[200/1875], Loss :0.2994
Epoch [2/5], Step[300/1875], Loss :0.3813
Epoch [2/5], Step[400/1875], Loss :0.1189
Epoch [2/5], Step[500/1875], Loss :0.3531
Epoch [2/5], Step[600/187

In [13]:
model.eval()

correct = 0 
total = 0
for img, label in test_loader:
    out=model(img)
    _, predicted = torch.max(out.data,1)
    total+=label.size(0)
    correct +=(predicted==label).sum().item()
    
print('Test Accuracy of the model on the 10000 test images : {} %'.format(100*correct/total))
torch.save(model.state_dict(),'model_nn.ckpt')

Test Accuracy of the model on the 10000 test images : 92.35 %
